In [35]:
import io, os, re, sys
import numpy as np
import pandas as pd
import pytesseract
import fitz
import PyPDF2

# Table extraction: Docling(doc:https://docling-project.github.io/docling/), tabula

from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
from googleapiclient.discovery import build
from PIL import Image

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [80]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.1
)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=os.getenv("GOOGLE_API_KEY")
        ) 

In [ ]:
print(sys.version)
print(sys.executable)

In [ ]:
os.listdir('data/')

# Extract data from PDF

In [ ]:
q1_report_path = "data/raw/2025_1Q_LGES_Audit_Report_CONFS_en.pdf"

In [ ]:
docs = fitz.open(q1_report_path)
print(f"total pages: {len(docs)}")

### Docling test

In [ ]:
from docling.document_converter import DocumentConverter
from pathlib import Path

In [ ]:
# doc_converter = DocumentConverter()
conv_res = doc_converter.convert(q1_report_path)
doc_filename = conv_res.input.file.stem

In [ ]:
dfs = {}
for table_i, table in enumerate(conv_res.document.tables):
    table_df: pd.DataFrame = table.export_to_dataframe()
    # print(table_df.to_markdown())
    dfs[f"tabel_{table_i}"] = table_df

----

In [ ]:
print(docs, type(docs))
print(docs[0], type(docs[0]))

In [ ]:
def is_scanned_page(page):
    """
    If pages are covered with only images, run OCR.

    ??? What if half is filled with images and half with text?
    """
    text = page.get_text().strip()
    images = page.get_images()
    return len(images) > 0 and len(text) < 25 

def extract_text_ocr(page):
    pix = page.get_pixmap()
    img_data = pix.tobytes("png")
    img = Image.open(io.BytesIO(img_data))

    # text extraction using OCR
    text = pytesseract.image_to_string(img)
    return text

def clean_text(text):
    """Clean extracted text"""
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters but keep punctuation
    text = re.sub(r'[^\w\s\-.,;:!?()]', '', text)
    return text.strip()

In [ ]:
full_text = ""
for pg_i, page in enumerate(docs):
    if is_scanned_page(page):
        ocr_text = extract_text_ocr(page)
        if ocr_text.strip():
            page_text = ocr_text
        else:
            print(f"--------OCR failed------- at page:{pg_i}")
    else:
        page_text = page.get_text()
        
    page_text = clean_text(page_text)
    full_text += f"\n--- Page {pg_i + 1} ---\n{page_text}\n"
    
with open('data/processed/2025_1Q_LGES_Audit_Report_CONFS_en_fulltext.txt', 'w') as f:
   f.write(full_text) 

In [ ]:
# ??? Extract Table of content: So it's information can be leveraged somewher(ex: chunk by chapter, etc..)
# table_of_contents = doc[1].get_text()

# ??? Better way to extract table? and standardize schema across various reports from diff countries.
# table = doc[5].get_text() + "\n" + doc[6].get_text()

## when PDF contains incomprehensible tables

Unless we use Multi-modal embedding model, or extracting tables work perfectly (Images seem fine but if table in pdf is hard to comprehend, even for human) it is better to add another chain, that is, ask LLM to generate a summary then summary becomes your texts to embed (or could feed it directly).
- Even with Docling, if table inside pdf is not easily comprehensible (even to human), we should either add human in the loop or summarize using multimodal-models.

In [ ]:
# from google import genai
# client = genai.Client()
# myfile = client.files.upload(file=q1_report_path)
# myfile

In [ ]:
from google import genai
from google.genai import types
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

client = genai.Client()
system_message = """
You are a financial analyst who have been following 
LG energy solution for few years.
You are extremely knowlegeable about battery industry.
From given financial report of LG energy solution, 
Create a summary report that will help investors make decisions 
on when to buy on the stock market.
"""

# Upload file
file = client.files.upload(file=q1_report_path)
while file.state.name == 'PROCESSING':
    time.sleep(2)
    file = client.files.get(name=file.name)

# Create cache
# model = 'models/gemini-1.5-flash-latest'
# cache = client.caches.create(
#     model=model,
#     config=types.CreateCachedContentConfig(
#         display_name='Cached Content',
#         system_instruction=(system_message),
#         contents=[file],
#         ttl="300s",
#     )
# )

chain = fin_rep_anal_prmt | gemini

message = HumanMessage(content="")
response = chain.invoke({
    "company_name": 
})

In [ ]:
import base64
from langchain_core.messages import HumanMessage

pdf_bytes = open(q1_report_path, 'rb').read()
pdf_base64 = base64.b64encode(pdf_bytes).decode('utf-8')

message = HumanMessage(
    content=[
        {"type": "text", "text": "You are a financial analyst who have been following LG energy solution for few years. You are extremely knowlegeable about battery industry. From given financial report of LG energy solution, Create a summary report that will help investors make decisions on when to buy on the stock market."},
        {
            "type": "file",
            "source_type": "base64",
            "mime_type":"application/pdf",
            "data": pdf_base64
        }
    ]
)

response = gemini.invoke([message])

In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


llm_with_tools = gemini.bind_tools([GetWeather, GetPopulation])
ai_msg = llm_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

In [ ]:
ai_msg.content

In [ ]:
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
resp = llm.invoke(
    "When is the next total solar eclipse in US?",
    tools=[GenAITool(google_search={})],
)

# Youtube extractor

In [ ]:
import requests
import google.auth

try:
    credentials, project = google.auth.default()
    print("✅ ADC credentials found and configured.")
    if project:
        print(f"Project ID: {project}")
    else:
        print("No project ID found in credentials.")
except google.auth.exceptions.DefaultCredentialsError as e:
    print(f"❌ ADC credentials not found. Error: {e}")

In [ ]:
"""
Sending video to LLM, despite it being multi-modal, is prone to error. It does not seem to capture 
information from video.

Better to use transcript whenever possible.
"""
from youtube_transcripts import fetch_transcripts_for_urls, merge_transcripts_text

urls = [
    # paste your video URLs here
]

trs = fetch_transcripts_for_urls(urls, preferred_languages=("en", "ko"), translate_to=None)
merged_text = merge_transcripts_text(trs)
print(merged_text[:1000])  # preview first 1,000 chars


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound, VideoUnavailable
from youtube_transcripts import fetch_transcript

In [54]:
def search_youtube_latest_company_news(client, company_name, language, k=15, days=10, n_sub_limit=0):
    """return list of youtube urls of latest company news
    https://developers.google.com/youtube/v3/docs/search/list#examples
    """
    # Only retrieve videos after this date
    pub_after_dt = datetime.now(timezone.utc) - timedelta(days=days)
    pub_after = pub_after_dt.isoformat().replace("+00:00", "Z")

    if language == "ko":
        query = f"{company_name} 기업 관련뉴스"
    else:
        query = f"{company_name} corporate news"
    # Exclude YouTube Shorts (videos under 60 seconds) by using the 'videoDuration' parameter set to 'medium' or 'long'.
    # 'medium' = 4-20min, 'long' = >20min. We'll use 'medium' to avoid shorts and most very short clips.
    # If you want to include longer videos too, you can run two queries or use 'any' and filter after.
    resp = (client.search()
            .list(q=query,
                 part="id,snippet",
                 type="video",
                 order="date",
                 maxResults=k,
                 publishedAfter=pub_after,
                 videoDuration="medium" # {"short"(0-4min), "medium"(4-20min), "long"(>20min)} 
            )).execute()  
    video_ids = []
    video_titles = []
    channel_ids = []
    channel_titles = []
    pub_dtts = []
    n_subs_list = []

    for res in resp.get("items"):
        content_type = res['id']['kind']
        if content_type != 'youtube#video':
            continue
        snippet = res['snippet']
        channel_id = snippet['channelId']
        channel_title = snippet['channelTitle']
        published_dtt = snippet['publishTime']
        
        video_id = res['id']['videoId'] # https://www.youtube.com/watch?v={video_id}
        video_title = res['snippet']['title']
        n_subs = (client.channels()
                .list(part='statistics',id=channel_id)
                .execute()['items'][0]['statistics']['subscriberCount'])
        video_ids.append(video_id)
        video_titles.append(video_title)
        channel_ids.append(channel_id)
        channel_titles.append(channel_title)
        pub_dtts.append(published_dtt)
        n_subs_list.append(int(n_subs))
    df = pd.DataFrame({"videoId":video_ids,
                    "title":video_titles,
                    "channelId":channel_ids,
                    "channelTitle":channel_titles,
                    "publishedAt":pub_dtts,
                    "subscriberCount":n_subs_list})
    df = df.loc[df["subscriberCount"] > n_sub_limit]
    df['videoUrl'] = df['videoId'].apply(lambda x: f"https://www.youtube.com/watch?v={x}")
    return df, resp

youtube_client = build("youtube", "v3")

company_name = "LG에너지 솔루션"
language = "ko"
k = 30
days = 10
n_sub_limit = 100_000

df, resp = search_youtube_latest_company_news(youtube_client, company_name, language, k, days, n_sub_limit)
urls_text = "\n".join(df['videoUrl'])

In [62]:
def get_transcript_text(ytt_api, video_id):
    transcription = ytt_api.fetch(video_id, languages=['ko', 'en'])
    transcript = ""
    for snippet in transcription.snippets:
        transcript += snippet.text + " "
    return transcript

df['transcript'] = df['videoId'].apply(lambda video_id: get_transcript_text(ytt_api, video_id))

In [ ]:
transcription = ytt_api.fetch(video_id, languages=['ko', 'en'])

In [63]:
df

,videoId,title,channelId,channelTitle,publishedAt,subscriberCount,videoUrl,transcript
0,E34kWPr8alA,[LIVE] 미국 조지아 소재 우리 기업 배터리공장 수색 관련/외교부 긴급 브리핑/...,UCcQTRi69dsVYHN3exePtZ1A,KBS News,2025-09-05T10:36:48Z,3420000,https://www.youtube.com/watch?v=E34kWPr8alA,네. 바로 시작하도록 하겠습니다. 미국 이민세관 단속국은 현지 시간 9월 4일 목요...
1,AIXFmdzpzDg,[마켓A컷] &quot;2차전지의 날&quot; 대형 계약 터지네!,UC8Sv6O3Ux8ePVqorx8aOBMg,이데일리TV,2025-09-04T07:45:42Z,326000,https://www.youtube.com/watch?v=AIXFmdzpzDg,준비한 사진은요.이 배터리와 관련된 사진을 저희가 준비를 했습니다. 오늘 2차전지 ...


In [66]:
from prompt_templates import youtube_news_summary_prompt

In [78]:
videos_block = [
    {
        "url": row["videoUrl"],
        "published_at": row["publishedAt"],
        "transcript": row.get("transcript", "")
    }
    for _, row in df.iterrows()
]

In [76]:
s = youtube_news_summary_prompt.format_messages(
    company_name=company_name,
    videos_block=videos,
    target_language=language,
)

In [77]:
print(s[0].content)


You are a financial news analyst and political analyst for individual equity investors.

Inputs:
- Company (optional): LG에너지 솔루션
- Output language: ko  # "en" for English, "ko" for Korean
- Videos (one or more entries with URL, PublishedAt, Transcript text; Title optional):
[{'url': 'https://www.youtube.com/watch?v=E34kWPr8alA', 'published_at': '2025-09-05T10:36:48Z', 'transcript': '네. 바로 시작하도록 하겠습니다. 미국 이민세관 단속국은 현지 시간 9월 4일 목요일 미조지아주에 소재한 우리 기업의 배터리 공장 건설 현장을 단속하였으며 그 과정에서 다수에 우리 국민이 구금되었습니다. 미국의 법 집행 과정에서 우리 투자 업체의 경제 활동과 우리 국민의 권위 부당하게 침해되어서는 안 될 것입니다.이 사건에 대해 주미국 대사관 총영사와 주 애틀랜타 총영사관의 영사를 현장에 급화하고 현지 공간을 중심으로 현장 대책 반을 다시 하겠습니다.이 이 사건에 대해 주미국 대사관 총사와 주 애런타 총영사관의 영사를 현장에 급화하고 현지 공관을 중심으로 현장 대책반을 출범시킬 것을 지시하는 등 적극 대처 중입니다. 서울에서도 오늘 주한 미국 대사관을 통해 우리의 우려와 유감의 뜻을 전달하고 우리 국민의 정당한 권위 치해당하지 않도록 각별히 유의해 줄 것을 당부하였습니다. 네. 이상입니다. 네. 바로 시작하도록 하겠습니다. 미국 이민세관 단속국은 현지 시간 9월 4일 목요일 미조지아주에 소재한 우리 기업의 배터리 공장 건설 현장을 단속하였으며 그 과정에서 다수에 우리 국민이 구금되었습니다. 미국의 법 집행 과정에서 우리 투자 업체의 경제 활동과 우리 국민의 권위 부당하게 침해되어서

LG 에너지 솔루션 기업정보 를 검색 해서 여러개의 동영상을 본 결과 전부 조자아주에서 일하는 한국인들 구금 된거 밖에 안나오는데<br>
요약 해달라고 하면 전혀 그런 얘기를 이해하지 못하거나/일부러 없애는것 같다.

예:
`https://www.youtube.com/watch?v=E34kWPr3alA` 의 경우 외교부장관이 미국 조지아 주에서 배터리공장 털린거 얘기만! 하는데도 "LG에너지솔루션이 북미 시장 선점과 IRA 수혜를 통해 다가오는 배터리 빅 사이클에서 강력한 성장 동력을 확보하고 있음을 강조하며 긍정적인 투자 관점을 제시합니다." 이라고 하고 responase에 이 관련 얘기는 하나도 없다.


내생각엔 영상 처리 못해서 default로 그냥 검색 한듯... ChatGPT나 Claude UI에선 제대로 작동.

In [81]:
chain = youtube_news_summary_prompt | gemini

print("number of videos: ", len(df['videoUrl']))
response = chain.invoke({
    "company_name": company_name,
    "videos_block": videos_block,
    "target_language": language
})

number of videos:  2


In [82]:
print(response.content)

## LG에너지솔루션 투자 분석: 미국 공장 단속, 기술 혁신 및 시장 동향

### 1) Executive Summary

최근 LG에너지솔루션과 관련된 주요 이슈는 미국 조지아주 배터리 공장 건설 현장에서 발생한 미국 이민세관 단속국의 단속 사건입니다. 이 과정에서 다수의 한국 국민이 구금되었으며, 한국 정부는 이에 대해 적극적으로 대응하고 있습니다. 이 사건은 LG에너지솔루션(또는 관련 한국 기업)의 현지 사업 운영에 단기적인 불확실성을 야기할 수 있습니다.

한편, 2차전지 산업 전반은 최근 수출 감소와 ETF 수익률 하락 등으로 부진한 모습을 보이고 있습니다. 그러나 LG에너지솔루션은 KAIST와 공동으로 12분 급속 충전 기술을 개발했다는 소식과 메르세데스-벤츠와 약 15조원 규모의 초대형 계약을 체결했다는 긍정적인 소식을 발표했습니다.

단기적으로는 미국 공장 단속 관련 불확실성과 시장 전반의 부진이 투자 심리에 부담으로 작용할 수 있으나, 중장기적으로는 기술 리더십 확보와 대규모 수주를 통한 안정적인 성장 동력이 기대됩니다.

### 2) Dated Key Points

*   **2025년 9월 5일 (정치/규제):** 미국 이민세관 단속국(ICE)이 9월 4일(현지시간 목요일) 미 조지아주에 소재한 한국 기업의 배터리 공장 건설 현장을 단속하여 다수의 한국 국민이 구금되었습니다. 한국 정부는 주미국 대사관 총영사와 주 애틀랜타 총영사관 영사를 현장에 급파하고 현장 대책반을 출범시키는 등 적극 대처 중입니다. 서울에서도 주한 미국 대사관을 통해 우려와 유감의 뜻을 전달하고 한국 국민의 정당한 권익 침해 방지를 당부했습니다. (해당 기업명은 명시되지 않았으나, 조지아주에 대규모 배터리 공장을 건설 중인 한국 기업 중 LG에너지솔루션이 유력하게 거론될 수 있습니다.)
*   **2025년 9월 4일 (산업/기업):**
    *   **시장 동향:** 최근 1주일간 국내 2차전지 관련 ETF(예: 코덱스 2차전지 레버리지)가 16%가량 하락하는 등

In [83]:
with open("data/processed/2025_09_11_LGES_news_summary.txt", "w", encoding="utf-8") as f:
    f.write(response.content)

## Sending video or youtube to Gemini

In [ ]:
video_bytes = open("/path/to/your/video.mp4", 'rb').read()
video_base64 = base64.b64encode(video_bytes).decode('utf-8')

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe what's in this video in a sentence"},
        {
            "type": "file",
            "source_type": "base64",
            "mime_type": "video/mp4",
            "data": video_base64
        }
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content

message = HumanMessage(
    content=[
        {"type": "text", "text": "summarize the video in 3 sentences."},
        {
            "type": "media",
            "file_uri": "https://www.youtube.com/watch?v=9hE5-98ZeCg",
            "mime_type": "video/mp4",
        }
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content